---
title: "Desafio 60K 2025 - Otimização de Rotação"
subtitle: "Análise de revezamento para canoa havaiana OC6"
author: "Outrigger Optimizer"
date: today
format:
  html:
    toc: true
    toc-depth: 3
    code-fold: true
    theme: cosmo
execute:
  warning: false
---


## Introdução

Este documento apresenta a análise de otimização de rotação para o **Desafio 60K 2025**.
O objetivo é encontrar a duração ideal de cada turno (período de remada) que minimize o tempo total de prova, considerando:

- **Fadiga**: remadores perdem eficiência ao longo de turnos consecutivos
- **Trocas**: cada troca de tripulação adiciona tempo à prova
- **Elegibilidade**: cada remador só pode ocupar determinados bancos



## Configuração da Prova


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from outrigger_opt import optimize_stint_range, solve_rotation_cycle

# Configuração da prova
DISTANCIA_KM = 60
VELOCIDADE_KMH = 10
TEMPO_TROCA_SECS = 40

### Tripulação

A tripulação é composta por 9 remadores, onde 6 remam e 3 descansam a cada turno.


In [ ]:
paddlers = pd.DataFrame({
    'name': ['Eduardo', 'Guilherme', 'Vitor', 'Ricardo', 'Airton',
             'Everson', 'Sergio', 'Marcelo', 'Zé']
})

# Habilidade relativa de cada remador (1.0 = média)
habilidade = [1.02, 1.0, 1, 1, 1, 1.02, 1, 1.01, 1.02]

# Peso de cada remador (kg) - usado para cálculo de trim (balanco)
peso = [73, 89, 78, 95, 95, 75, 75, 97, 78]

# Tabela da tripulação
tripulacao = pd.DataFrame({
    'Nome': paddlers['name'],
    'Habilidade': [f'{h:.0%}' for h in habilidade],
    'Peso (kg)': peso
})
tripulacao

### Matriz de Elegibilidade

Cada remador só pode ocupar determinados bancos, baseado em sua experiência e posição preferida.


In [ ]:
#| label: tbl-elegibilidade
#| tbl-cap: 'Matriz de Elegibilidade (1 = pode sentar, 0 = não pode)'

eligibility = np.array([
    [1, 1, 0, 0, 0, 0],  # Eduardo
    [1, 1, 0, 0, 0, 0],  # Guilherme
    [1, 1, 1, 1, 1, 1],  # Vitor
    [0, 1, 1, 1, 1, 0],  # Ricardo
    [0, 0, 1, 1, 1, 0],  # Airton
    [1, 1, 1, 1, 1, 0],  # Everson
    [0, 0, 0, 1, 1, 0],  # Sergio
    [0, 0, 1, 1, 1, 1],  # Marcelo
    [1, 1, 1, 1, 1, 1],  # Zé
])

# Criar DataFrame para visualização
df_elig = pd.DataFrame(
    eligibility,
    index=paddlers['name'],
    columns=['Banco 1 (Voga)', 'Banco 2', 'Banco 3', 'Banco 4', 'Banco 5', 'Banco 6 (Leme)']
)
df_elig.replace({1: '✓', 0: ''})

In [ ]:
#| fig-cap: "Mapa de elegibilidade dos remadores"

# Calculate average and std of eligible seat positions for each paddler
seat_positions = np.arange(1, 7)  # 1-6
paddler_stats = []
for i, name in enumerate(paddlers['name']):
    eligible_seats = seat_positions[eligibility[i] == 1]
    avg_seat = eligible_seats.mean()
    std_seat = eligible_seats.std() if len(eligible_seats) > 1 else 0
    paddler_stats.append((i, name, avg_seat, std_seat))

# Sort by average (increasing), then by std (decreasing)
paddler_stats.sort(key=lambda x: (x[2], -x[3]))
sorted_indices = [s[0] for s in paddler_stats]
sorted_names = [s[1] for s in paddler_stats]
sorted_eligibility = eligibility[sorted_indices]

fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(sorted_eligibility, cmap='Blues', aspect='auto')

ax.set_xticks(range(6))
ax.set_xticklabels(['Banco 1 (Voga)', 'Banco 2', 'Banco 3', 'Banco 4', 'Banco 5', 'Banco 6 (Leme)'])
ax.set_yticks(range(9))
ax.set_yticklabels(sorted_names)

# Adicionar texto nas células
for i in range(9):
    for j in range(6):
        text = '✓' if sorted_eligibility[i, j] == 1 else ''
        ax.text(j, i, text, ha='center', va='center', fontsize=14)

ax.set_title('Matriz de Elegibilidade', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Justificativa dos Parâmetros

### Padrão de Entrada nos Bancos

A rotação tradicional (equipes experientes) segue o padrão:

- **Voga**: Remador descansado entra no **banco 1 (voga)** e passa pro 2 na troca seguinte — assim o voga está sempre "fresco" (lá ele)

Para a equipe **Caveiras**, invertemos esse padrão:

::: {.callout-note}
## Adaptação para Caveiras
**Voga**: Entrar no banco 2 primeiro, esquentar, depois ir pro banco 1.

*Justificativa*: O atleta vai subir pro banco 1 provavelmente frio, dificilmente acertará o ritmo de imediato. Melhor entrar no 2, esquentar e acompanhar o voga anterior, e só depois ir pro 1.

**Leme**: Marcelo entra no banco 5, depois vai pro 5. Zé direto no 6.
:::

### Pesos de Entrada (`seat_entry_weights`)

Os pesos refletem a preferência de entrada em cada banco (maior = preferido para entrada):


In [ ]:
seat_entry_weights = [1, 2, 1.5, 1.5, 2, 1]
seat_names = ['Banco 1 (Voga)', 'Banco 2', 'Banco 3', 'Banco 4', 'Banco 5', 'Banco 6 (Leme)']

pd.DataFrame({
    'Banco': seat_names,
    'Peso': seat_entry_weights
}).set_index('Banco')

### Duração do turno


## Otimização

Testamos durações de turno entre **10 e 20 minutos** para encontrar o tempo ótimo.


In [ ]:
#| output: false

results = optimize_stint_range(
    paddlers,
    stint_km_range=[1,1.5,2],
    seat_eligibility=eligibility,
    seat_weights=[1.05, 1.02, 1.02, 1.01, 1.00, 1.10],
    seat_entry_weights=seat_entry_weights,
    paddler_ability=habilidade,
    paddler_weight=peso,
    trim_penalty_weight=0.75,
    moi_penalty_weight=0.25,
    distance_km=DISTANCIA_KM,
    speed_kmh=VELOCIDADE_KMH,
    switch_time_secs=TEMPO_TROCA_SECS,
    max_consecutive=3,
    solver_time_secs=60,
    gap_tolerance=0.001,
)

### Comparação de Tempos


In [ ]:
#| label: fig-comparacao
#| fig-cap: Tempo de prova vs. duração do turno

summary = results['summary']

fig, ax = plt.subplots(figsize=(10, 6))

# Gráfico principal
ax.plot(summary['stint_min'], summary['race_time'], 'b-o', linewidth=2, markersize=8)

# Destacar o melhor
best_idx = summary['race_time'].idxmin()
best_stint = int(summary.loc[best_idx, 'stint_min'])
best_time = summary.loc[best_idx, 'race_time']

ax.scatter([best_stint], [best_time], color='red', s=200, zorder=5, label=f'Ótimo: {best_stint} min')
ax.axhline(y=best_time, color='red', linestyle='--', alpha=0.5)
ax.axvline(x=best_stint, color='red', linestyle='--', alpha=0.5)

ax.set_xlabel('Duração do Turno (minutos)', fontsize=12)
ax.set_ylabel('Tempo Total de Prova (minutos)', fontsize=12)
ax.set_title('Otimização da Duração do Turno', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Adicionar anotação
ax.annotate(f'{best_time:.1f} min\n({best_time/60:.1f} horas)',
            xy=(best_stint, best_time),
            xytext=(best_stint + 2, best_time + 3),
            fontsize=11,
            arrowprops=dict(arrowstyle='->', color='red'))

plt.tight_layout()
plt.show()

### Tabela de Resultados


In [ ]:
#| label: tbl-resultados
#| tbl-cap: Comparação de tempos por duração de turno

summary_display = summary.copy()
summary_display['race_time_hours'] = summary_display['race_time'] / 60
summary_display['speed'] = DISTANCIA_KM / summary_display['race_time_hours']
summary_display.columns = ['Turno (min)', 'Nº turnos', 'Output Médio', 'Tempo (min)', 'Tempo (horas)', 'Velocidade (km/h)']
summary_display['Output Médio'] = summary_display['Output Médio'].apply(lambda x: f'{x:.1%}')
summary_display['Tempo (horas)'] = summary_display['Tempo (horas)'].apply(lambda x: f'{x:.2f}')
summary_display['Velocidade (km/h)'] = summary_display['Velocidade (km/h)'].apply(lambda x: f'{x:.2f}')
summary_display = summary_display.drop(columns=['Tempo (min)'])
summary_display.set_index('Turno (min)', inplace=True)
summary_display

### Trade-off: Trocas vs. Fadiga


In [ ]:
#| label: fig-tradeoff
#| fig-cap: Análise do trade-off entre número de trocas e output médio

fig, ax1 = plt.subplots(figsize=(10, 6))

color1 = 'tab:blue'
ax1.set_xlabel('Duração do Turno (minutos)', fontsize=12)
ax1.set_ylabel('Número de Trocas', color=color1, fontsize=12)
ax1.plot(summary['stint_min'], summary['n_stints'] - 1, color=color1, marker='s', linewidth=2, label='Trocas')
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()
color2 = 'tab:orange'
ax2.set_ylabel('Output Médio (%)', color=color2, fontsize=12)
ax2.plot(summary['stint_min'], summary['avg_output'] * 100, color=color2, marker='o', linewidth=2, label='Output')
ax2.tick_params(axis='y', labelcolor=color2)

# Linha vertical no ótimo
ax1.axvline(x=best_stint, color='red', linestyle='--', alpha=0.5, label=f'Ótimo: {best_stint} min')

fig.suptitle('Trade-off: Menos Trocas vs. Mais Fadiga', fontsize=14, fontweight='bold')
fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.02), ncol=3)
plt.tight_layout()
plt.subplots_adjust(bottom=0.15)
plt.show()

## Resultado Ótimo

::: {.callout-tip}
## Configuração Recomendada
**Duração do turno: `{python} best_stint` minutos**

Tempo estimado de prova: **`{python} f'{best_time:.1f}'` minutos** (`{python} f'{best_time/60:.1f}'` horas)
:::

### Regras de Rotação

Cada remador segue um padrão simples de 3 turnos que se repete durante toda a prova:


In [ ]:
print("Ciclo de Rotação (repete a cada 3 turnos):\n")
for name, rule in results['best']['cycle_rules'].items():
    print(f"  {name:12} {rule}")

### Ciclo de Rotação

O cronograma segue um ciclo de **3 turnos que se repete** durante toda a prova:


In [ ]:
#| label: tbl-schedule
#| tbl-cap: Ciclo de rotação (turnos 1-3 se repetem)

schedule = results['best']['schedule'].copy()
schedule.columns = schedule.columns.str.strip()

# Show 6 rows (2 full cycles) to make pattern obvious
n_rows = min(6, len(schedule))
cycle = schedule.head(n_rows).copy()
cycle.index = [f'Turno {i+1}' for i in range(n_rows)]
cycle.columns = ['Banco 1', 'Banco 2', 'Banco 3', 'Banco 4', 'Banco 5', 'Banco 6']
cycle

In [ ]:
#| label: tbl-cycle-stats
#| tbl-cap: Estatísticas por turno do ciclo

# Calculate per-stint statistics
params = results['best']['parameters']
trim_stats = params.get('trim_stats')
seat_positions = trim_stats['seat_positions'] if trim_stats else [-2.5, -1.5, -0.5, 0.5, 1.5, 2.5]
name_to_weight = dict(zip(paddlers['name'], peso))
name_to_ability = dict(zip(paddlers['name'], habilidade))
seat_weights = [1.05, 1.02, 1.02, 1.01, 1.00, 1.10]

cycle_stats = []
for t in range(3):  # cycle length = 3
    row = results['best']['cycle_schedule'].iloc[t]

    # Calculate weighted power output (sum of ability * seat_weight)
    power = sum(name_to_ability.get(name, 1.0) * seat_weights[s] for s, name in enumerate(row))

    # Calculate trim moment
    trim = sum(name_to_weight.get(name, 75) * seat_positions[s] for s, name in enumerate(row))

    # Speed is proportional to power (simplified)
    base_speed = VELOCIDADE_KMH
    speed = base_speed * (power / sum(seat_weights))  # Normalize by ideal power

    cycle_stats.append({
        'Turno': f'{t+1}',
        'Output': f'{power:.2f}',
        'Velocidade (km/h)': f'{speed:.2f}',
        'Trim (kg-m)': f'{trim:+.1f}',
    })

pd.DataFrame(cycle_stats).set_index('Turno')

In [ ]:
#| label: fig-heatmap
#| fig-cap: Ciclo de rotação (turnos 1-3 se repetem)
from matplotlib.patches import Rectangle

# Show 6 rows (2 full cycles)
n_rows = min(6, len(results['best']['schedule']))
schedule_matrix = results['best']['schedule'].head(n_rows).copy()
all_paddlers = set(paddlers['name'])

# Build matrix with paddlers out (resting)
out_matrix = []
for i in range(n_rows):
    in_canoe = set(schedule_matrix.iloc[i].values)
    out = sorted(all_paddlers - in_canoe)
    out_matrix.append(out)
out_df = pd.DataFrame(out_matrix, columns=['Fora 1', 'Fora 2', 'Fora 3'])

# Combine in and out (9 columns total)
combined = pd.concat([schedule_matrix.reset_index(drop=True), out_df], axis=1)
paddler_to_num = {name: i for i, name in enumerate(paddlers['name'])}
combined_numeric = combined.replace(paddler_to_num)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), gridspec_kw={'width_ratios': [6, 3], 'wspace': 0.05})

# Left plot: seats (in canoe)
im1 = ax1.imshow(combined_numeric.iloc[:, :6].values, cmap='tab10', aspect='auto')
ax1.set_xticks(range(6))
ax1.set_xticklabels(['Banco 1\n(Voga)', 'Banco 2', 'Banco 3', 'Banco 4', 'Banco 5', 'Banco 6\n(Leme)'])
ax1.set_yticks(range(n_rows))
ax1.set_yticklabels([f'Turno {i+1}' for i in range(n_rows)])

for i in range(n_rows):
    for j in range(6):
        name = combined.iloc[i, j]
        ax1.text(j, i, name[:3], ha='center', va='center', fontsize=10, fontweight='bold', color='white')

# Highlight first cycle
rect = Rectangle((-0.5, -0.5), 6, 3, linewidth=3, edgecolor='yellow', facecolor='none', linestyle='--')
ax1.add_patch(rect)

# Right plot: out (resting) - same colors as left
im2 = ax2.imshow(combined_numeric.iloc[:, 6:].values, cmap='tab10', aspect='auto')
ax2.set_xticks(range(3))
ax2.set_xticklabels(['Fora 1', 'Fora 2', 'Fora 3'])
ax2.set_yticks(range(n_rows))
ax2.set_yticklabels([])

for i in range(n_rows):
    for j in range(3):
        name = combined.iloc[i, 6 + j]
        ax2.text(j, i, name[:3], ha='center', va='center', fontsize=10, fontweight='bold', color='white')

fig.suptitle(f'Padrão de Rotação (ciclo de 3 turnos repete {results["best"]["n_stints"] // 3}x)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Análise de Trim e Concentração de Peso

O modelo considera dois aspectos do balanço da canoa:

- **Trim (momento)**: Balanço proa-popa. Positivo = pesada na popa, negativo = pesada na proa.
- **MOI (inércia)**: Concentração de peso. Menor = peso no centro, maior = peso nas pontas.


In [ ]:
#| label: tbl-trim
#| tbl-cap: Análise de trim e MOI por turno do ciclo

trim_stats = results['best'].get('parameters', {}).get('trim_stats')
if trim_stats:
    print(f"Momento de trim médio (absoluto): {trim_stats['avg_abs_trim_moment']:.1f} kg-m")
    print(f"Momento de trim máximo (absoluto): {trim_stats['max_abs_trim_moment']:.1f} kg-m")
    print(f"MOI médio: {trim_stats.get('avg_moi', 0):.1f} kg-m²")
    print(f"\nPor turno do ciclo:")
    moi_values = trim_stats.get('moi_values', [0] * len(trim_stats['trim_moments']))
    for t, (m, moi) in enumerate(zip(trim_stats['trim_moments'], moi_values)):
        direcao = "popa" if m > 0 else "proa" if m < 0 else "neutro"
        print(f"  Turno {t+1}: Trim={m:+.1f} kg-m ({direcao}), MOI={moi:.1f} kg-m²")
else:
    print("Análise não disponível (trim_penalty_weight = 0 e moi_penalty_weight = 0)")

## Resumo


In [ ]:
n_stints = results['best']['n_stints']
n_trocas = n_stints - 1
velocidade_efetiva = DISTANCIA_KM / (best_time / 60)  # km/h
subidas_por_remador = n_stints / 3  # cada remador entra 1x por ciclo de 3 turnos

| Métrica | Valor |
|---------|-------|
| Distância | `{python} DISTANCIA_KM` km |
| Velocidade base | `{python} VELOCIDADE_KMH` km/h |
| **Velocidade efetiva** | **`{python} f'{velocidade_efetiva:.2f}'` km/h** |
| Duração do turno | `{python} best_stint` min |
| Número de turnos | `{python} n_stints` |
| Número de trocas | `{python} n_trocas` |
| Subidas por remador | `{python} f'{subidas_por_remador:.1f}'` (média) |
| Tempo por troca | `{python} TEMPO_TROCA_SECS` s |
| **Tempo total** | **`{python} f'{best_time:.1f}'` min** (`{python} f'{best_time/60:.1f}'` h) |

---

*Gerado com [Outrigger Optimizer](https://github.com/anthropics/outrigger)*